# Periodic frequent-pattern discovery
Using algorithm from this paper: https://www.researchgate.net/publication/356825094_Discovering_Periodic-Frequent_Patterns_in_Uncertain_Temporal_Databases?enrichId=rgreq-1de5ac4c085dd4a641f85dda9c527a37-XXX&enrichSource=Y292ZXJQYWdlOzM1NjgyNTA5NDtBUzoxMTQzMTI4MTA4MzQ2MzY1M0AxNjYyNTk3MzM3NTUz&el=1_x_3&_esc=publicationCoverPdf

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import numpy as np
from tqdm import tqdm
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import sortednp
import datetime
import json
from operator import itemgetter

# Input

## Read the data

In [ ]:
database_path = r"../data/UTDATABASE/utd_20221222_0226/label.csv"
database_df = pd.read_csv(database_path)
database_df.fillna(0, inplace=True)
# database_df.describe()

# Preprocess

## Handle time

I assume hour can contribute to the traffic jam?

In [ ]:
database_df['Datetime'] = pd.to_datetime(database_df['Datetime'], errors='coerce')
first_datetime = database_df['Datetime'].min()
database_df['Time'] = np.round((database_df['Datetime'] - first_datetime).dt.total_seconds() / 3600)
database_df.sort_values(by=['Time'], ignore_index=True, inplace=True)

Note: there is a time gap that is 120 hours in between, which I suspect is because of data loss. It make all the features have at least their period >= 120. Which I don't think is a good 'bug'. So I planned to delete that time gap. Even though it could lead to data loss, I hope the remaining data can still achieve good patterns. (There's still plenty of days to calculate.)

In [ ]:
#debug
time_val_counts = database_df['Time'].value_counts()
debug_time_df = database_df[ ['Time'] ] 
debug_time_df['Diff'] = debug_time_df.diff()
delete_time_start, max_time_gap = debug_time_df['Time'][ debug_time_df['Diff'].idxmax() ], debug_time_df['Diff'][ debug_time_df['Diff'].idxmax() ]
print(delete_time_start, max_time_gap)

In [ ]:
# danger! delete time gap
database_df = database_df[ database_df['Time'] >= delete_time_start ]
database_df.reset_index(drop=True, inplace=True)

In [ ]:
datetime_series = database_df.Datetime

deleted_columns = ['SensorCode', 'Datetime' ]
#database_df.drop([ 'dt' ], axis=1, inplace=True)
database_df.drop([ col for col in database_df if col in deleted_columns ], axis=1, inplace=True)

Copy into another database for further use.

In [ ]:
database_df_copy = database_df.copy(True)

### Delete time related labels

Because I think it won't really contribute much.

In [ ]:
database_df.drop( [ col for col in database_df if ('HourTriple' in col) or ('WeekDay' in col) ], axis=1, inplace=True )

# Algorithm

## Find frequent items (1-pattern)

Both sum of prob and max of time difference (period)

In [ ]:
exclude_columns = ['Time']

In [ ]:
def prepare_database(database_df,
                    min_support, max_period):
    expsup_one_df = database_df.sum(axis=0)

    period_one_df = expsup_one_df.copy(deep=True)
    min_time, max_time = database_df['Time'].min(), database_df['Time'].max()
    for col in database_df.columns:
        if (col == 'Time'): continue
        ser = database_df['Time'][ database_df[col] > 0 ]
        #print(ser.min(), ser.diff().max(), ser.max())
        period_one_df[col] = max( ser.min() - min_time, ser.diff().max(), max_time - ser.max() )

    pf_one_items = [ [col, expsup_one_df[col], period_one_df[col]] 
                        for col in database_df.columns 
                        if (col != 'Time') and (expsup_one_df[col] >= min_support) and (max_period >= period_one_df[col])  ]
    pf_one_df = pd.DataFrame(pf_one_items, columns = ['Item', 'ExpSup', 'MaxPeriod'])
    pf_one_df.sort_values(by=['ExpSup', 'MaxPeriod'], ignore_index=True, ascending=False, inplace=True)
    pf_one_df

    items = pf_one_df['Item'].unique().tolist()
    mining_df = database_df[ items + ['Time'] ]
    return mining_df, items, pf_one_df, min_time, max_time

## Convert the table into new format for algorithm

Transaction = [ times, ('item', prob) ]. times = Timestamp

In [ ]:
def make_transactions(mining_df):
    transactions = [ None for i in range(mining_df.shape[0]) ]
    for i in tqdm(range(mining_df.shape[0])):
        item_list = [ ]
        for col in mining_df.columns:
            if (col == 'Time'): continue
            if (mining_df.loc[i, col] > 0):
                item_list.append( (col, mining_df.loc[i, col]) )
        transactions[i] = [ mining_df.loc[i, 'Time'], item_list ]
    return transactions

## Define tree

Some of the code are taken from: https://github.com/Likhitha-palla/UPFP

In [ ]:
class Node(object):
    def __init__(self, item, children, parent, probability=0):
        self.item = item
        self.probability = probability              # expSupCap of the path/pattern
        self.children = children
        self.parent = parent
        #self.times = []
        self.times = np.ndarray(shape=(0))

    def addChild(self, node):
        if (node.item not in self.children):
            self.children[node.item] = []
        self.children[node.item] = node
        node.parent = self

In [ ]:
class Tree(object):
    def __init__(self, items):
        self.root = Node('Root Node', {}, None)
        self.items = items
        self.nodelists = {}     # item  : [ nodes of this item ]
        for i in items:
            self.nodelists[i] = []
        

## Build tree

In [ ]:
def add_transaction(tree: Tree, time, itemlist):
    node = tree.root
    maxprob = 1
    for i in range(len(itemlist)):
        item, prob = itemlist[i]         
        if (item in node.children):
            node = node.children[item]
        else:
            new_node = Node(item, {}, node)
            node.addChild(new_node)
            node = new_node
            tree.nodelists[item].append(node)
        node.probability += prob*maxprob                # multiply by maximum prob of previous items in this transaction.
        maxprob = min(maxprob, prob)                    # changing to min, see if it's better
    node.times = np.append( node.times, time)

In [ ]:
def build_tree(transactions, items):
    tree = Tree(items)    
    for trans in transactions:
        add_transaction(tree, trans[0], trans[1])
    return tree

### Remove item from tree entirely

In [ ]:
def remove_item(tree: Tree, item):
    for node in tree.nodelists[item]:
        node.parent.times = np.concatenate([node.parent.times, node.times])
        
        #sortednp.merge( node.parent.times, node.times ) 
        #node.parent.times += node.times
        node.parent.children[item] = None
        del node

## UPFP-growth

The list of patterns shall look like this: [ [ items ], support, period ]

The string produced for report for each pattern: {items} [support, period]

For example: "AQI_O3_MED, Motorbike_MED" [245, 154]

In [ ]:
def traverse_path(node: Node):
    path = []
    p = node
    while (p.parent.item != 'Root Node'):
        path.append(p.parent.item)
        p = p.parent
    path.reverse()
    return path

In [ ]:
def find_period(times: np.ndarray, min_time, max_time):
    if (times is None) or (len(times) == 0): return 9999999
    ts = np.sort(times)
    period = max( min(ts) - min_time, max_time - max(ts) )
    if (len(ts) > 1):
        period = max(period, np.max(np.diff(ts)))
    # for i in range(len(ts)-1):
    #     difference = ts[i+1] - ts[i]
    #     period = max(period, difference)
    return period

In [ ]:
def find_frequent_items(cond_patterns, cond_ts, cond_sups, min_support, max_period, min_time, max_time):
    item_sup_per = { }
    for i in range(len(cond_patterns)):
        for item in cond_patterns[i]:
            if (item not in item_sup_per):  
                item_sup_per[item] = [ 0 , np.ndarray(shape=(0)) ]          # support/prob, period
            item_sup_per[item][0] += cond_sups[i]
            item_sup_per[item][1] = np.concatenate( [ item_sup_per[item][1], cond_ts[i] ] )  # appending lists
    for item in item_sup_per:
        item_sup_per[item][1] = find_period(item_sup_per[item][1], min_time, max_time)
    freq_item_dict = { key: value for key, value in item_sup_per.items() if (value[0] >= min_support) and (value[1] <= max_period) }
    freq_item_dict = dict(sorted(freq_item_dict.items(), key=lambda item: (item[1][0], item[1][1])))

    return freq_item_dict

In [ ]:
def make_condition_pattern_base(tree: Tree, item, min_support, max_period, min_time, max_time):
    cond_patterns = [ None for i in range(len(tree.nodelists[item])) ]
    cond_times = [ None for i in range(len(tree.nodelists[item])) ]
    cond_sups = np.ndarray(shape=(len(tree.nodelists[item])))
    #[ None for i in range(len(tree.nodelists[item])) ]
    
    i = 0
    for node in tree.nodelists[item]:
        cond_patterns[i] = traverse_path(node)
        cond_times[i] = node.times 
        cond_sups[i] = node.probability
        i += 1
    freq_item_sup_per = find_frequent_items(cond_patterns, cond_times, cond_sups, min_support, max_period, min_time, max_time)
    
    new_patterns, new_times, new_sups = [], [], np.ndarray(shape=(len(tree.nodelists[item])))
    count = 0
    for p in cond_patterns:
        p1 = [ item for item in p if item in freq_item_sup_per ]
        if (len(p1) > 0):
            p1 = sorted(p1, key=lambda item: (freq_item_sup_per[item][0], freq_item_sup_per[item][1]), reverse=True )
            new_patterns.append(p1)
            new_times.append(cond_times[count])
            #new_sups.append(cond_sups[count])
            new_sups[count] = cond_sups[count]
        count += 1
    new_sups = new_sups[0 : count]
    return freq_item_sup_per, new_patterns, new_times, new_sups

In [ ]:
def add_transaction_condition(tree: Tree,transaction,times,sup):
    node=tree.root
    for item in transaction:
        if item not in node.children:
            new_node=Node(item,{}, node)
            node.addChild(new_node)
            # if item not in tree.nodelists:
            #     tree.nodelists[ item ] = []
            tree.nodelists[item].append(new_node)            
        node = node.children[item] 
        node.probability += sup           
    node.times = np.concatenate( [node.times, times] )

In [ ]:
# return the list of all pattern satisfying the constrains. 
def upfp_growth(tree: Tree, prefix, 
                min_support, max_period, 
                min_time, max_time):
    mined_patterns = []

    items_list = tree.items
    items_list.reverse()            # start from the least frequent item, which have the leaf nodes
    for item in items_list:
        newprefix = prefix + [item]
        expsup = 0
        for node in tree.nodelists[item]:
            expsup += node.probability
        if (expsup >= min_support):
            freq_item_sup_per, cond_patterns, cond_times, cond_sups = make_condition_pattern_base(tree, item, 
                                                                                min_support, max_period, 
                                                                                min_time, max_time)
            cond_tree = Tree(list(freq_item_sup_per.keys()))
            for p in range(len(cond_patterns)):
                add_transaction_condition(cond_tree, cond_patterns[p], cond_times[p], cond_sups[p])
            if (len(cond_patterns) > 0):
                mined_patterns += upfp_growth(cond_tree, newprefix, min_support, max_period, min_time, max_time ) 
            else:
                # if no more items to search: stop and return.
                mined_patterns.append(newprefix)
        remove_item(tree, item)
    return mined_patterns

## Recalculate actual Expected support for one more time

In [ ]:
def produce_patterns(mined_patterns, mining_df, min_time, max_time, min_support, max_period):

    max_pattern_length = 0
    for ptn in mined_patterns:
        max_pattern_length = max(max_pattern_length, len(ptn))
    print('Potential max pattern length: ', max_pattern_length)

    filtered_patterns = []
    for ptn in mined_patterns:
        if ( len(ptn) < (max_pattern_length - 3) ): continue
        pattern_df = mining_df[ ptn ]

        pattern_df['Prob'] = 1
        for col in pattern_df.columns:
            if (col == 'Prob') or (col == 'Time'): continue
            pattern_df['Prob'] *= pattern_df[col]
        expSup = pattern_df.Prob.sum()
        
        time_series = mining_df['Time'][  pattern_df['Prob'] > 0 ] 
        period = 0
        if (time_series is None) or (time_series.shape[0] == 0): 
            period = 999999
        else:
            period = max( time_series.min() - min_time, max_time - time_series.max() )
            
        if (len(time_series) > 1):
            period = max( period, time_series.diff().max() )
        
        if (expSup >= min_support) and (period <= max_period):
            #print(expSup, ' ', period)
            filtered_patterns.append( [ ptn, expSup, period ] )

    max_pattern_length = 0
    for ptn in filtered_patterns:
        max_pattern_length = max(max_pattern_length, len(ptn[0]))

    final_patterns = []
    for ptn in filtered_patterns:
        if (len(ptn[0]) == max_pattern_length):
            final_patterns.append(ptn)

    del filtered_patterns

    # sort
    final_patterns.sort( key = lambda p: (len(p[0]), -p[2], p[1]), reverse=True )
    print('Max pattern length: ', max_pattern_length)

    pattern_strings = []
    for ptn in final_patterns:
        item_str = ""
        for i in range(len(ptn[0]) - 1):
            item_str += ptn[0][i] + ', '
        item_str += ptn[0][ len(ptn[0]) - 1 ]
        item_str += ':[' + str(ptn[1]) + ', ' + str(ptn[2]) + ']\n' 
        pattern_strings.append( item_str )
    
    print('Pattern generated.')
    return pattern_strings, final_patterns

## Export results

In [ ]:
def export_results(output_folder_path: str, 
                pattern_strings, pf_one_df, database_path, min_support, max_period):
    pf_one_df.to_csv(os.path.join(output_folder_path,'fp_pattern_one.csv'))
    with open(os.path.join(output_folder_path, 'patterns.txt'), 'w') as f:
        f.writelines(pattern_strings)
        f.close()
    with open(os.path.join(output_folder_path, 'setting.json'), 'w') as f:
        f.write( json.dumps({ 'data_path': database_path,  'min_support': min_support, 'max_period': max_period }, indent=4) )

# Actual run

With sensor + traffic features.

In [ ]:
def mine_patterns(database_df, output_folder_path, min_support, max_period):
    os.makedirs(output_folder_path, exist_ok=True)
    mining_df, item_list, pf_one_df, min_time, max_time = prepare_database(database_df, min_support, max_period)
    transactions = make_transactions(mining_df)
    tree = build_tree(transactions, item_list)
    mined_patterns = upfp_growth(tree, [], min_support, max_period, min_time, max_time )
    pattern_strings, _ = produce_patterns(mined_patterns, mining_df, min_time, max_time, min_support, max_period)
    export_results(output_folder_path, pattern_strings, pf_one_df, database_path, min_support, max_period)

In [ ]:
min_default_support = 500
max_default_period = 1000

In [ ]:
output_folder_path = '../output_temp/FULLDATA/ufp_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
mine_patterns(database_df, output_folder_path, min_default_support, max_default_period)

# Custom mining for other purpose

## Define constants

In [ ]:
sensor_columns = ['Altitude', 'Temperature', 'Humidity', 'Rainfall', 'WindGust', 'WindSpeed', 'WindCos', 'WindSin', 'UV']
traffic_columns = ['Person', 'Car', 'Bus', 'Motorbike', 'Truck']

In [ ]:
def delete_columns(database_df, columns):
    column_names = []
    for c in database_df.columns:
        for l in columns:
            if l in c:
                column_names.append(c)
                break
    #print(column_names)
    new_database_df = database_df.drop( column_names, axis=1 ).reset_index(drop=True)
    #new_database_df.info()
    return new_database_df

## Mining with only traffic data

In [ ]:
min_traffic_support = 10
max_traffic_period = 1000

In [ ]:
output_traffic_folder_path = r'../output_temp/TRAFFIC/ufp_traffic_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_traffic_folder_path, exist_ok=True)

traffic_database_df = delete_columns(database_df, sensor_columns)
mine_patterns(traffic_database_df, output_traffic_folder_path,
            min_traffic_support, max_traffic_period)
# del traffic_database_df

## Mining with only sensor data

In [ ]:
# output_sensor_folder_path = '../data/UTDatabase/Mining/ufp_sensor_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
# os.makedirs(output_sensor_folder_path, exist_ok=True)

# sensor_database_df = delete_columns(database_df, traffic_columns)
# mine_patterns(sensor_database_df, output_sensor_folder_path, min_support, max_period)
# del sensor_database_df

# Customized mining

## Mining with specific columns for traffic relation finding

This is to see if for extreme cases such as PM10 lv4, is there a hidden pattern among them?
Each columns shall be selected, and the dataframe will be filtered based on that col.

In [ ]:
min_spec_support = 3
max_spec_period = 2000

In [ ]:
traffic_database_df = delete_columns(database_df, sensor_columns)

In [ ]:
# exclude_unhelpful_columns = ['CO', 'NO2', 'SO2']
# dropping_unhelpful_columns = []
# for col in traffic_database_df.columns:
#     for ex in exclude_unhelpful_columns:
#         if (ex in col) :
#             dropping_unhelpful_columns.append(col)
# new_traffic_database_df = traffic_database_df.drop( dropping_unhelpful_columns , axis=1).reset_index(drop=True)

In [ ]:
# for col in new_traffic_database_df.columns:
#     if ('HIGH' in col) or ('lv4' in col) or ('lv5' in col) or ('HourTriple' in col): 
#         sub_df = new_traffic_database_df[ new_traffic_database_df[col] > 0 ].copy(True).reset_index(drop=True)
#         output_col_path = os.path.join(output_speccol_path, col)
#         mine_patterns(sub_df, output_col_path, min_spec_support, max_spec_period )
#         del sub_df

## Mining with each one AQI item

For each iteration, isolate one AQI item from the rest (delete all other AQI items), to find pattern specifically for that item.

In [ ]:
min_AQI_support = 1
max_AQI_period = 2000

In [ ]:
aqi_label_columns = [ col for col in traffic_database_df.columns if 'AQI' in col ]
aqi_label_columns

aqi_df = traffic_database_df[ aqi_label_columns ].copy(True)

custom_aqi_database_df = traffic_database_df.drop( aqi_label_columns, axis=1 )
custom_aqi_database_df = custom_aqi_database_df.drop( [ c for c in custom_aqi_database_df.columns if 'WeekDay' in c ], axis=1 )

In [ ]:
output_aqi_path = r"../output_temp/AQI/ufp_aqi_" + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_aqi_path, exist_ok=True)

for aqicol in aqi_label_columns:
    print(aqicol)
    sub_df = custom_aqi_database_df.copy(True)
    sub_df[aqicol] = aqi_df[aqicol].copy(True)
    sub_df = sub_df[ sub_df[aqicol] > 0 ].sort_values(by=['Time'], ignore_index=True)               # prob = 0 mean not exist, dont use
    if (sub_df.shape[0] == 0): continue
    output_col_path = os.path.join(output_aqi_path, aqicol)
    mine_patterns(sub_df, output_col_path, min_AQI_support, max_AQI_period )
    del sub_df

## Mining with only high traffic signals

In [ ]:
min_hightraffic_support = 15
max_hightraffic_period = 2000

In [ ]:
# high_traffic_database_df = traffic_database_df
# for col in traffic_columns:
#     high_traffic_database_df = high_traffic_database_df[ high_traffic_database_df[ col + '_LOW' ] == 0 ]
# high_traffic_database_df.reset_index(drop=True, inplace=True)

In [ ]:
# output_hightraffic_path = r"../output_temp/HIGHTRAFFIC/ufp_hightraffic_" + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
# os.makedirs(output_hightraffic_path, exist_ok=True)

# mine_patterns(high_traffic_database_df, output_hightraffic_path, min_hightraffic_support, max_hightraffic_period)

# Mining with each traffic item

The same process to each AQI item.

In [ ]:
min_eachtraffic_support = 1.5
max_eachtraffic_period = 1000

In [ ]:
traffic_label_columns = []
for tcol in traffic_columns:
    for col in traffic_database_df.columns:
        if (tcol in col):
            traffic_label_columns.append(col)

trafficonly_df = traffic_database_df[ traffic_label_columns ].copy(True)
traffic_excluded_database_df = traffic_database_df.drop( traffic_label_columns, axis=1 )
# traffic_excluded_database_df = traffic_excluded_database_df.drop( [ c for c in traffic_excluded_database_df.columns if 'WeekDay' in c ], axis=1 )

In [ ]:
traffic_excluded_database_df.describe()

In [ ]:
output_eachtraffic_path = r"../output_temp/EACHTRAFFIC/ufp_eachtraffic_" + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_eachtraffic_path, exist_ok=True)

for c in traffic_label_columns:
    print(c)
    sub_df = traffic_excluded_database_df.copy(True)
    sub_df[c] = trafficonly_df[c].copy(True)
    sub_df = sub_df[ sub_df[c] > 0 ].sort_values(by=['Time'], ignore_index=True)               # prob = 0 mean not exist, dont use
    if (sub_df.shape[0] == 0): continue
    output_col_path = os.path.join(output_eachtraffic_path, c)
    mine_patterns(sub_df, output_col_path, min_eachtraffic_support, max_eachtraffic_period )
    del sub_df